# Predecir si los clientes se irán o se quedarán de Beta Bank
Ayudaremos a Beta Bank a predecir si los clientes se irán o se quedarán basándonos en diferentes datos del cliente como su Puntuación de Crédito, Ubicación Geográfica, Sexo, Edad, cuánto tiempo lleva su deposito en el banco, Saldo, Número de Productos que utilizan, si tienen tarjeta de crédito, si son miembros activos y su Salario Estimado. Esta información se ha recogido en un conjunto de datos que vamos a analizar mediante modelos predictivos.

# Contents <a id='back'></a>

* [Etapa 1. Descripción de los datos](#data_review)
* [Etapa 2. Data preprocessing](#data_preprocessing)
    * [2.1 Eliminación de algunas columnas](#column_elimination)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Dummies para las columnas categóricas](#dummies)
    * [2.4 Escala de las columnas numéricas](#scale_columns)
    * [2.5 Características, objetivo y división de los datos](#features_target_segment)
    * [2.7 Tener en consideración el desequilibrio de clases](#to_have_class_imbalance)
* [Etapa 3. Examinar el equilibrio de clases](#class_balance)
    * [3.1 Construcción de un modelo con el desequilibrio de clases](#class_imbalance)
    * [3.2 Tener en consideración el desequilibrio de clases](#to_have_class_imbalance)
    * [3.3 Ajuste del peso(weight) de la clase](#Class_weight_setting)
    * [3.4 Sobremuestreo](#Upsampling)
* [Etapa 4. Finalmente haremos pruebas en el conjunto de pruebas](#final_test)
* [Conclusiones](#end)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

## Información General

Importar bibliotecas y módulos necesarios

In [4]:
import pandas as pd #for dealing with dataframes
from sklearn.tree import DecisionTreeClassifier #to deal with Decision Tree Models
from sklearn.ensemble import RandomForestClassifier #to deal with Random Forest Models
from sklearn.linear_model import LogisticRegression #to deal with Logistic Regression Models
from sklearn.model_selection import train_test_split #to be able to split datasets
from sklearn.preprocessing import StandardScaler #to be able to scale values
from sklearn.utils import shuffle #to be able to shuffle columns
from sklearn.metrics import f1_score, roc_auc_score #to be able to calculate model's f1_score
#from sklearn.metrics import  #to be able to calculate model's auc-roc
import numpy as np

Cargaremos el archivo para leer nuestro dataframe

In [5]:
try:
    data = pd.read_csv('C:/Users/USER/Documents/proyectos/proyecto 8 (terminado)/Churn.csv')
except:
    data = pd.read_csv('/datasets/Churn.csv')

Características
-
- `RowNumber`: índice de cadena de datos
- `CustomerId`: identificador de cliente único
- `Surname`: apellido
- `CreditScore`: valor de crédito
- `Geography`: país de residencia
- `Gender`: sexo
- `Age`: edad
- `Tenure`: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- `Balance`: saldo de la cuenta
- `NumOfProducts`: número de productos bancarios utilizados por el cliente
- `HasCrCard`: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- `IsActiveMember`: actividad del cliente (1 - sí; 0 - no)
- `EstimatedSalary`: salario estimado

Objetivo
-
- `Exited`: El cliente se ha ido (1 - sí; 0 - no)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [7]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


- Nuestro objetivo(**"target"**) será la columna **"Exited"**, mientras que las demás columnas servirán como características(**"features"**). 
- En primer lugar, tenemos que tratar los valores que faltan ausentes en la columna **"Tenure"**. 
- En segundo lugar, tenemos que crear variables ficticias para las columnas categóricas.
- A continuación, tendremos que normalizar (escalar) las columnas numéricas, excepto las que tienen valores binarios (1 ó 0) como **"HasCrCard"** e **"IsActiveMember"**.

[Volver a Contenidos](#back)

---

## Etapa 2. Procesamiento de datos <a id='data_preprocessing'></a>

*Nombres de columnas en minúsculas
Utilizaremos el método 'str.lower()

In [8]:
#esto cambia los nombres de las columnas a minúsculas y guarda los cambios
data.columns = data.columns.str.lower()

#mostraremos de nuevo la info para ver los cambios efectuados
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           9091 non-null   float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


[Volver a Contenidos](#back)

### Eliminación de algunas columnas <a id='column_elimination'></a>

Eliminación de algunas columnas
- Las columnas en cuestión son **"rownumber"**, **"customerid"**, y **"surname"**. RowNumber es básicamente el índice, sólo que empieza por 1 y no por 0. CusttomerId es sólo para diferenciar a los clientes con un numero único, Surname es también otro medio de identificación; ambos son diferentes para cada observación. Incluir estas columnas no ayudará con el entrenamiento de nuestros modelos. Así que tenemos que eliminar estas columnas

In [9]:
df=data.drop(['rownumber', 'customerid', 'surname'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   creditscore      10000 non-null  int64  
 1   geography        10000 non-null  object 
 2   gender           10000 non-null  object 
 3   age              10000 non-null  int64  
 4   tenure           9091 non-null   float64
 5   balance          10000 non-null  float64
 6   numofproducts    10000 non-null  int64  
 7   hascrcard        10000 non-null  int64  
 8   isactivemember   10000 non-null  int64  
 9   estimatedsalary  10000 non-null  float64
 10  exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


Hemos eliminado las columnas éxitosamente y procederemos a tratar los valores asuentes de la columna Tenure.

[Volver a Contenidos](#back)

---

### Valores ausentes <a id='missing_values'></a>

Tratar Valores ausentes de la columna Tenure 

In [10]:
df.head()

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [11]:
df["tenure"].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., nan])

In [12]:
variable = df.isna().sum()
print(variable)

creditscore          0
geography            0
gender               0
age                  0
tenure             909
balance              0
numofproducts        0
hascrcard            0
isactivemember       0
estimatedsalary      0
exited               0
dtype: int64


In [13]:
#esto agrupa los 'datos' por la columna "model_year" y "condition", y calcula la mediana de las lecturas del odometer
#y con el metodo .to_dict() que convierte el dataframe resultante en un diccionario con el nombre de la variable: median_mileage
median_mileage = df.groupby(["creditscore"])['tenure'].median().to_dict()

def odometer_missing_values(row):
    if np.isnan(row['tenure']): 
        return median_mileage.get(row['creditscore'])
    return row['tenure']
    
#aplica la función odometer_missing_values a la columna "odometer" de nuestro dataframe para efectuar los cambios
df['tenure'] = df.apply(odometer_missing_values, axis=1)

In [14]:
variable = df.isna().sum()
print(variable)

creditscore        0
geography          0
gender             0
age                0
tenure             2
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
exited             0
dtype: int64


Podemos rellenar estas celdas vacías con el valor medio de la columna Tenure para no introducir ningún sesgo en nuestro conjunto de datos

In [15]:
#rellena las celdas que faltan con la mediana
df['tenure']=df['tenure'].fillna(df['tenure'].median())
df['tenure'].sort_values().unique()

array([ 0. ,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  5.5,
        6. ,  6.5,  7. ,  7.5,  8. ,  9. , 10. ])

In [16]:
variable = df.isna().sum()
print(variable)

creditscore        0
geography          0
gender             0
age                0
tenure             0
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
exited             0
dtype: int64


éxito todos los valores que faltaban en Tenure han sido sustituidos con la mediana

[Volver a Contenidos](#back)

---

### Dummies para las columnas categóricas <a id='dummies'></a>
Ahora tenemos 2 columnas categóricas: **"Geography"** y **"Gender"**. Vamos hacer un conteo de valores de cada columna

In [17]:
df['geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: geography, dtype: int64

In [18]:
df['gender'].value_counts()

Male      5457
Female    4543
Name: gender, dtype: int64

- Para la columna Geography, tenemos 3 valores: Francia, Alemania y España. Cuando categorizamos esta columna, la columna será reemplazada por 3 columnas: Geography_in_France, Geography_in_Germany y Geography_in_ Spain. Cada columna tomará el valor 1 en la observación en la que la columna Geografía tenía el país como valor, de lo contrario obtiene 0. 
- Será lo mismo para la columna Género. Utilizaremos la función pd.get_dummies en toda la tabla 'df' ya que son las únicas columnas categóricas. Podemos eliminar una de las columnas ficticias para ambos escenarios porque un 0 en España y Alemania, por ejemplo, implica directamente un 1 para Francia. Para ello, configuremos el parámetro drop_first=True.

In [19]:
#sustituye las columnas categóricas por sus ficticias y elimina la primera columna ficticia de cada columna sustituida
df=pd.get_dummies(df, drop_first=True)
df.head()

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Hemos creado con éxito los dummies para las columnas Geography y Gender

[Volver a Contenidos](#back)

---

### Escala de las columnas numéricas <a id='scale_columns'></a>
Nuestras columnas numéricas son: **'creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary'**. Estas variables no tienen un rango definido, por lo que necesitamos escalarlas (o estandarizarlas) obteniendo sus puntuaciones z-scores. Hacemos esto porque el algoritmo normalmente pensaría que las variables con alta dispersión son más importantes y no queremos eso. Así que llamaremos a nuestra función StandardScaler(), tambien utilizaremos el metofo fit() para ajustar las columnas numéricas en ella y las transformarlas, entonces obtendremos nuestros valores escalados

In [20]:
numeric = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary']
scaler = StandardScaler()
scaler.fit(data[numeric])
df[numeric] = scaler.transform(df[numeric])
df.head()

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_Germany,geography_Spain,gender_Male
0,-0.326221,0.293517,-1.035627,-1.225848,-0.911583,1,1,0.021886,1,0,0,0
1,-0.440036,0.198164,-1.381103,0.117350,-0.911583,0,1,0.216534,0,0,1,0
2,-1.536794,0.293517,1.037224,1.333053,2.527057,1,0,0.240687,1,0,0,0
3,0.501521,0.007457,-1.381103,-1.225848,0.807737,0,0,-0.108918,0,0,0,0
4,2.063884,0.388871,-1.035627,0.785728,-0.911583,1,1,-0.365276,0,0,1,0


Hemos escalado correctamente los valores de las columnas numéricas de dataframe

[Volver a Contenidos](#back)

---

### Características, objetivo y división de los datos <a id='features_target_segment'></a>
- objetivo será sin duda la columna **"Exited"**, mientras que el resto de columnas serán las características(**"features"**). Tenemos que dividir ambos conjuntos en conjuntos de entrenamiento, validación y prueba, que constituyen el 60%, el 20% y el 20% respectivamente.
- Para ello, llamaremos a la función train_test_split() dos veces. La primera vez, dividiremos el conjunto de entrenamiento y un segundo conjunto estableciendo el parámetro test_size=0,4 (que es el porcentaje del conjunto de datos que debe constituir el segundo conjunto). La segunda vez, dividiremos el segundo conjunto anterior en tamaños iguales (test_size=0,5) y los resultados serán el conjunto de validación (20% del conjunto de datos original) y el conjunto de prueba (20% del conjunto de datos original). El random_state se igual  establecerá en 12345 y lo mantendremos igual durante todo el entrenamiento del modelo.

In [21]:
features=df.drop('exited', axis=1)
target=df['exited']

In [22]:
features.describe()

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,geography_Germany,geography_Spain,gender_Male
count,1.000000e+04,1.000000e+04,10000.000000,1.000000e+04,1.000000e+04,10000.00000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,-4.824585e-16,2.318146e-16,-0.000031,-6.252776e-17,1.634248e-17,0.70550,0.515100,-2.877698e-17,0.250900,0.247700,0.545700
std,1.000050e+00,1.000050e+00,0.960214,1.000050e+00,1.000050e+00,0.45584,0.499797,1.000050e+00,0.433553,0.431698,0.497932
min,-3.109504e+00,-1.994969e+00,-1.726578,-1.225848e+00,-9.115835e-01,0.00000,0.000000,-1.740268e+00,0.000000,0.000000,0.000000
25%,-6.883586e-01,-6.600185e-01,-0.690152,-1.225848e+00,-9.115835e-01,0.00000,0.000000,-8.535935e-01,0.000000,0.000000,0.000000
50%,1.522218e-02,-1.832505e-01,0.000798,3.319639e-01,-9.115835e-01,1.00000,1.000000,1.802807e-03,0.000000,0.000000,1.000000
75%,6.981094e-01,4.842246e-01,0.691748,8.199205e-01,8.077366e-01,1.00000,1.000000,8.572431e-01,1.000000,0.000000,1.000000
max,2.063884e+00,5.061197e+00,1.728174,2.795323e+00,4.246377e+00,1.00000,1.000000,1.737200e+00,1.000000,1.000000,1.000000


In [23]:
target.describe()

count    10000.000000
mean         0.203700
std          0.402769
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: exited, dtype: float64

In [24]:
#primera división para obtener conjuntos de entrenamiento tanto para las características como para el objetivo (60%) y un segundo conjunto (40%)
features_train, features_test_valid, target_train, target_test_valid=train_test_split(features, target, test_size=0.4, random_state=12345)
#2ª división del segundo conjunto anterior en los conjuntos de validación y prueba, división uniforme
features_valid, features_test, target_valid, target_test=train_test_split(features_test_valid, target_test_valid, test_size=0.5, random_state=12345)
#imprime las longitudes de los 3 conjuntos de características y objetivos que derivamos de la división
print(len(features_train), len(target_train), len(features_valid), len(target_valid), len(features_test), len(target_test))

6000 6000 2000 2000 2000 2000


Particionamos correctamente nuestras características(**"features"**) y objetivos(**"target"**), y dividimos los datos en conjuntos de entrenamiento(**"train"**), validación(**"valid"**) y prueba(**"test"**) con sus proporciones correctas

[Volver a Contenidos](#back)

---

## Etapa 3. Examinar el equilibrio de clases <a id='class_balance'></a>

### Construcción de un modelo con el desequilibrio de clases <a id='class_imbalance'></a>

**Clasificación del Árbol de Decisión**
- Vamos a utilizar la función DecisiontreeClassifier(). llamaremos 2 hiperparámetros: **"random_state"** y **"max_depth"**. random_state tiene que ser el mismo en todos los ámbitos por lo que le daremos un valor fijo (12345). max_depth, sin embargo, es el hiperparámetro que vamos a iterar. Así que haremos un **"bucle for"**  para max_depth (en este caso del 1 al 10) y obtendremos sus puntuaciones F1 y valores AUC-ROC, que son métricas de la calidad del modelo.
- El F1_score procesa el objetivo del conjunto de validación y las predicciones. La función roc_auc_score procesa el objetivo del conjunto de validación con las probabilidades de clase positivas de cada observación del conjunto válido. Para ello utilizamos la función predict_proba()

In [25]:
for i in range(1, 11):
    dt_model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    dt_model.fit(features_train, target_train)
    dt_pred_valid=dt_model.predict(features_valid)
    probabilities_valid = dt_model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    #imprimeremos la puntuación F1 comparando las predicciones con el objetivo del conjunto de validación y
    # la puntuación auc_roc comparando el objetivo del conjunto de validación con las probabilidades de clase positivas.
    print('Max depth', i, 'F1 score =', f1_score(target_valid, dt_pred_valid), 'AUC-ROC score =', \
         roc_auc_score(target_valid, probabilities_one_valid))

Max depth 1 F1 score = 0.0 AUC-ROC score = 0.6925565119556736
Max depth 2 F1 score = 0.5217391304347825 AUC-ROC score = 0.7501814673449512
Max depth 3 F1 score = 0.4234875444839857 AUC-ROC score = 0.7973440741838507
Max depth 4 F1 score = 0.5528700906344411 AUC-ROC score = 0.813428129858032
Max depth 5 F1 score = 0.5406249999999999 AUC-ROC score = 0.8221680508592478
Max depth 6 F1 score = 0.5696969696969697 AUC-ROC score = 0.8164631712023421
Max depth 7 F1 score = 0.5320813771517998 AUC-ROC score = 0.8090154489199669
Max depth 8 F1 score = 0.5343511450381679 AUC-ROC score = 0.8084438267833703
Max depth 9 F1 score = 0.5694249649368863 AUC-ROC score = 0.7822255760075976
Max depth 10 F1 score = 0.53954802259887 AUC-ROC score = 0.7631231134957264


El mejor **"F1_score (~0,57)"** se observa en **"max_depth 6"**, con un valor **"AUC-ROC de ~0,82"**.

**Clasificación Bosque Aleatorio**
- Utilizaremos la función RandomForestClassifier(). Nuestro hiperparámetro random_state seguira siendo el mismo. Los hiperparámetros con los que iteraremos son max_depth y n_estimators. En este caso crearemos primero una lista vacía. A continuación, haremos un bucle for a través de los valores de profundidad máxima y, dentro de ese bucle, otro bucle a través de los valores de n_estimadores. Utilizaremos este bucle para crear modelos con diferentes permutaciones de los valores de max_depth y n_estimators que almacenaremos en la lista, de la que elegiremos el modelo con la puntuación f1 más alta.

In [26]:
rf = []
for i in range(1, 11):
    for j in range(10, 101, 10):
        rf_model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j)
        rf_model.fit(features_train, target_train)
        rf.append(rf_model)
    
print(max(rf, key=lambda rf_model: f1_score(rf_model.predict(features_valid), target_valid)))
#prints the model from the list with the highest f1 score based on predictions made using the 
#features of the validation set and the actual target of the validation set

RandomForestClassifier(max_depth=10, n_estimators=60, random_state=12345)


El modelo RandomForestClassifier con la puntuación F1 más alta tiene unos hiperparámetros **"max_depth=10"** y **"n_estimators=10"**. Así que vamos a entrenarlo específicamente con esos hiperparámetros y obtener un F1_score y un roc_auc_score. similar a la anterior.

In [27]:
best_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10)
best_rf_model.fit(features_train, target_train)
best_rf_pred = best_rf_model.predict(features_valid)
probabilities_rf_valid=best_rf_model.predict_proba(features_valid)
probabilities_rf_one_valid=probabilities_rf_valid[:, 1]
print('F1 score =', f1_score(target_valid, best_rf_pred), 'AUC-ROC =', \
      roc_auc_score(target_valid, probabilities_rf_one_valid))

F1 score = 0.58493353028065 AUC-ROC = 0.8386475541226357


La mejor puntuación F1 es ~0,59, con una puntuación AUC-ROC de ~0,85

**Regresión logística**
- Utilizaremos la función LogisticRegression(). Nuestro random_state se mantendra el mismo. Los hiperparámetros max_depth y n_estimators no se aplican aquí. Todo lo que necesitamos es establecer un solucionador. Usaremos  **solver='liblinear**

In [28]:
lr_model = LogisticRegression(random_state=12345, solver='liblinear')
lr_model.fit(features_train, target_train)
lr_valid_pred=lr_model.predict(features_valid)
probabilities_lr_valid=lr_model.predict_proba(features_valid)
probabilities_lr_one_valid=probabilities_lr_valid[:, 1]
print('F1 score =', f1_score(target_valid, lr_valid_pred), 'AUC-ROC =', roc_auc_score(target_valid, probabilities_lr_one_valid))

F1 score = 0.33108108108108103 AUC-ROC = 0.7588677042566191


**Conclusión Intermedia**
- Observando los tres modelos podemos ver que el mejor de los 3 modelos fue el clasificador Random Forest con hiperparámetros **"max_depth=10"** y **"n_estimators=10"**, ya que obtuvo la puntuación F1 (aproximadamente 0,59) y la puntuación AUC-ROC (aproximadamente 0,84) más altas. Lo utilizaremos en el proximamente.

[Volver a Contenidos](#back)

---

### Tener en consideración el desequilibrio de clases <a id='to_have_class_imbalance'></a>
- Analizaremos el desequilibrio de clases para conocer las porciones de cada clase en el objetivo(target) del dataframe de entrenamiento. Para ello, utilizaremos la función value_counts() y estableceremos el parámetro **"normalize=True"**.

In [29]:
target_train.value_counts(normalize=True)
#shows unique values of target_train and their shares (percentages) of the data

0    0.800667
1    0.199333
Name: exited, dtype: float64

- clase negativa (0) representa ~80% de los datos, mientras que la clase positiva (1) representa ~20 de los datos. Por tanto, hay 4 veces más en la clase negativa0 que en la clase positiva 1. 
- Veremos dos enfoques para analizar el desequilibrio de clases.

[Volver a Contenidos](#back)

### balance clase del peso "class_weight=balance" <a id='Class_weight_setting'></a>
- Para abordar este punto lo que tenemos que hacer aquí es establecer el hiperparámetro class_weight='balanced' al entrenar el modelo. Esto hará que la clase más rara (1 en este caso) tenga más peso. Aparte de eso, la sintaxis es la misma que antes.

In [30]:
bal_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10, class_weight='balanced')
bal_rf_model.fit(features_train, target_train)
bal_rf_pred = bal_rf_model.predict(features_valid)
proba_bal_rf_valid=bal_rf_model.predict_proba(features_valid)
proba_bal_rf_one_valid=proba_bal_rf_valid[:, 1]
print('F1 score =', f1_score(target_valid, bal_rf_pred), 'AUC-ROC =', roc_auc_score(target_valid, proba_bal_rf_one_valid))

F1 score = 0.6130177514792899 AUC-ROC = 0.8395480858219564


La puntuación F1 ya es mejor que antes, pero el valor AUC-ROC sufrió un ligero descenso casi imperceptible. La tasa de verdaderos positivos seguramente disminuyó un poco.

In [31]:
bal_lr_model = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
bal_lr_model.fit(features_train, target_train)
bal_lr_valid_pred=bal_lr_model.predict(features_valid)
probabilities_bal_lr_valid=bal_lr_model.predict_proba(features_valid)
probabilities_bal_lr_one_valid=probabilities_bal_lr_valid[:, 1]
print('F1 score =', f1_score(target_valid, lr_valid_pred), 'AUC-ROC =', roc_auc_score(target_valid, probabilities_bal_lr_one_valid))

F1 score = 0.33108108108108103 AUC-ROC = 0.763895861939644


La puntuacion F1 es casi similar antes del balance, pero el valor AUC-ROC aumento un poco a 0.76

In [32]:
bal_dt_model = DecisionTreeClassifier(random_state=12345, max_depth=6, class_weight = 'balanced')
bal_dt_model.fit(features_train, target_train)
bal_dt_pred_valid=bal_dt_model.predict(features_valid)
probabilities_bal_valid = bal_dt_model.predict_proba(features_valid)
probabilities_bal_one_valid = probabilities_bal_valid[:, 1]
#imprimeremos la puntuación F1 comparando las predicciones con el objetivo del conjunto de validación y
# la puntuación auc_roc comparando el objetivo del conjunto de validación con las probabilidades de clase positivas.
print('F1 score =', f1_score(target_valid, bal_dt_pred_valid), 'AUC-ROC score =', roc_auc_score(target_valid, probabilities_bal_one_valid))

F1 score = 0.5533522190745988 AUC-ROC score = 0.7940753936329157


La puntuacion "F1_score (0,55)" se observa en "max_depth 6", con un valor "AUC-ROC de 0,79". en esta ocasion podemos ver que el balance que me hemos hecho se observa un decremento tanto para F1 como para "AUC-ROC"

[Volver a Contenidos](#back)

### Sobremuestreo(upsample) <a id='Upsampling'></a>
En este punto, repetiremos la clase más rara y sus observaciones suficientes veces para que coincida por igual con la otra clase. Hemos visto antes que hay 4 veces más 0 clase negativa que 1 clase positiva, así que repetiremos los 1 y sus observaciones 4 veces para igualar los 0 en el conjunto de entrenamiento. Después de hacerlo, tendremos que barajarlos utilizando la función shuffle().

In [33]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1] 
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

In [34]:
#pasaremos como argumento el conjunto de características de entrenamiento y el objetivo de entrenamiento en la función upsample con una repetición de 4
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
print(features_upsampled.shape, target_upsampled.shape)

(9588, 11) (9588,)


Con esto podemos entrenar nuestro modelo utilizando estas características y objetivos sobremuestreados.

In [35]:
ups_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10)
ups_rf_model.fit(features_upsampled, target_upsampled)
ups_rf_pred = ups_rf_model.predict(features_valid)
proba_ups_rf_valid=ups_rf_model.predict_proba(features_valid)
proba_ups_rf_one_valid=proba_ups_rf_valid[:, 1]
print('F1 score =', f1_score(target_valid, ups_rf_pred), 'AUC-ROC =', roc_auc_score(target_valid, proba_ups_rf_one_valid))

F1 score = 0.6082251082251082 AUC-ROC = 0.8391201253334463


La puntuación F1 es inferior a la obtenida al utilizar el ajuste por peso de clase. Sin embargo, ocurre lo contrario cuando se trata de AUC-ROC, mostrando de nuevo una ligera diferencia. La tasa de verdaderos positivos debe de haber aumentado.

In [36]:
ups_lr_model = LogisticRegression(random_state=12345, solver='liblinear')
ups_lr_model.fit(features_upsampled, target_upsampled)
bal_lr_pred=ups_lr_model.predict(features_valid)
proba_ups_lr_valid=ups_lr_model.predict_proba(features_valid)
proba_ups_lr_one_valid=proba_ups_lr_valid[:, 1]
print('F1 score =', f1_score(target_valid, bal_lr_pred), 'AUC-ROC =', roc_auc_score(target_valid, proba_ups_lr_one_valid))

F1 score = 0.49056603773584906 AUC-ROC = 0.7638036160392937


La puntuación F1 es superior "0.49" a la obtenida al utilizar el ajuste por peso de clase. Cuando se trata de AUC-ROC "0.76" se mantuvo casi igual.

In [37]:
ups_dt_model = DecisionTreeClassifier(random_state=12345, max_depth=6)
ups_dt_model.fit(features_upsampled, target_upsampled)
ups_dt_pred_valid=ups_dt_model.predict(features_valid)
proba_ups_valid = ups_dt_model.predict_proba(features_valid)
proba_ups_one_valid = proba_ups_valid[:, 1]
#imprimeremos la puntuación F1 comparando las predicciones con el objetivo del conjunto de validación y
# la puntuación auc_roc comparando el objetivo del conjunto de validación con las probabilidades de clase positivas.
print('F1 score =', f1_score(target_valid, ups_dt_pred_valid), 'AUC-ROC score =', roc_auc_score(target_valid, proba_ups_one_valid))

F1 score = 0.5533522190745988 AUC-ROC score = 0.7940753936329157


La puntuación F1 es igual "0.55" a la obtenida al utilizar el ajuste por peso de clase. Cuando se trata de AUC-ROC "0.79" se mantuvo igual.

**Conclusión Intermedia**
- Seguiremos adelante con el enfoque de ajuste del peso de la clase, ya que tiene la puntuación **"F1 más alta de 0,61"**.

[Volver a Contenidos](#back)

---

## Etapa 4. Finalmente haremos pruebas en el conjunto de pruebas <a id='final_test'></a>
- Vamos a aplicar nuestro modelo (con el ajuste del peso de las clases) al conjunto de prueba. 
- Antes debemos entrenar el modelo utilizando los conjuntos de entrenamiento y validación; los uniremos utilizando la metodo **"pd.concat()"**.

In [38]:
#features_train_final=pd.concat([features_train] + [features_valid])
#target_train_final=pd.concat([target_train] + [target_valid])
final_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10, class_weight='balanced')
final_rf_model.fit(features_train, target_train)
final_rf_pred = final_rf_model.predict(features_test)
proba_rf_test=final_rf_model.predict_proba(features_test)
proba_rf_one_test=proba_rf_test[:, 1]
print('F1 score =', f1_score(target_test, final_rf_pred), 'AUC-ROC =', roc_auc_score(target_test, proba_rf_one_test))

F1 score = 0.6014150943396226 AUC-ROC = 0.8434529457883793


La puntuación final de F1 es de 0,60, por encima del umbral de 0,59.

[Volver a Contenidos](#back)

# Conclusión General <a id='end'></a>

- Procesamos el conjunto de datos escalando a las columnas numéricas, rellenando los valores que faltaban en la columna **"Tenure"** y obteniendo columnas ficticias a partir de las categóricas. Segmentamos los datos, sin tener en cuenta el desequilibrio de clases de 4:1, entrenamos los modelos de **árbol de decisión, bosque aleatorio y regresión logística**, y determinamos que el **"modelo bosque aleatorio"** era el mejor debido a su elevada **"puntuación F1 (alrededor de 0,59)"** y un **"valor AUC-ROC de alrededor de 0,85"**. 
- Teniendo en cuenta el desbalance de clases añadimos el hiperparametro class_weight=balance y utilizamos dos enfoques: Ajuste del peso de la clase decidimos por el primero ya que presentaba la puntuación F1 más alta (0,61), 
- Entrenamos el modelo con datos de entrenamiento y validación y lo aplicamos al conjunto de pruebas, obteniendo una puntuación final de **"F1 de 0,60"**.